In [1]:

import requests
from PIL import Image
import pytesseract
from bs4 import BeautifulSoup
import re 
from datetime import datetime, timedelta


In [2]:
def request_first_page(fromdate,todate):
    # URL yêu cầu
    url = f'https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx?CertificateDateFrom={fromdate}&CertificateDateTo={todate}&Status=-1&FormCOId=0&CountryId=0&CertificateNumber=&CustomsNumber=&ReceiverName=&TransportMethodId=0'

    # Đặt headers
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        'Cookie': cookie,
        'DNT': '1',
        'Referer': 'https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx?',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"'
    }

    # Gửi yêu cầu GET
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        print(f"Request {fromdate},{todate} thành công!")
        return response.text
    # print(response2.text)  # Nội dung HTML
    else:
        print(f"Request thất bại: {response.status_code}")
        return None


In [3]:
def request_more_page(eventvalidation,viewstate,page_index): 
    import requests

    # URL cần truy vấn
    more_page_url = "https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx"

    # Các headers sử dụng trong request
    headers2 = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Cookie": cookie,
        "DNT": "1",
        "Origin": "https://ecosys.gov.vn",
        "Referer": "https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx?CertificateDateFrom=01/01/2024&CertificateDateTo=31/03/2024&Status=-1&FormCOId=0&CountryId=0&CertificateNumber=&CustomsNumber=&ReceiverName=&TransportMethodId=0",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
    }

    # Dữ liệu form gửi kèm
    data2 = {
        
        "__EVENTVALIDATION": eventvalidation,
        "__VIEWSTATE":viewstate,
        "ctl00$cplhContainer$ddlChoiceIndexOfPage":page_index
    }

    # Gửi POST request
    response2 = requests.post(more_page_url, headers=headers2, data=data2)

    # Kiểm tra kết quả trả về
    if response2.status_code == 200:
        print(f"Request page {page_index} thành công!")
        return response2.text
        # print(response2.text)  # Nội dung HTML
    else:
        print(f"Request thất bại: {response2.status_code}")
        return None


In [9]:
def generate_dates(start_date, end_date):
    """Tạo danh sách các quý theo từng giai đoạn lùi."""
    quarters = []
    current_start = datetime.strptime(start_date, "%d/%m/%Y")
    current_end = datetime.strptime(end_date, "%d/%m/%Y")
    
    while current_start >= datetime.strptime("01/01/2022", "%d/%m/%Y"):
        quarters.append((current_start.strftime("%d/%m/%Y"), current_end.strftime("%d/%m/%Y")))
        # Lùi lại một quý
        current_end = current_start - timedelta(days=1)
        current_start = (current_end.replace(day=1) - timedelta(days=1)).replace(day=1)
    quarters.append(("01/01/2022", current_end.strftime("%d/%m/%Y")))
    return quarters

# Ngày bắt đầu và kết thúc cho quý hiện tại
start_date = "01/10/2024"
end_date = (datetime.now() + timedelta(days=1)).strftime("%d/%m/%Y")

# Tạo danh sách các quý
dates_list = generate_dates(start_date, end_date)

In [10]:
dates_list

[('01/10/2024', '26/12/2024'),
 ('01/08/2024', '30/09/2024'),
 ('01/06/2024', '31/07/2024'),
 ('01/04/2024', '31/05/2024'),
 ('01/02/2024', '31/03/2024'),
 ('01/12/2023', '31/01/2024'),
 ('01/10/2023', '30/11/2023'),
 ('01/08/2023', '30/09/2023'),
 ('01/06/2023', '31/07/2023'),
 ('01/04/2023', '31/05/2023'),
 ('01/02/2023', '31/03/2023'),
 ('01/12/2022', '31/01/2023'),
 ('01/10/2022', '30/11/2022'),
 ('01/08/2022', '30/09/2022'),
 ('01/06/2022', '31/07/2022'),
 ('01/04/2022', '31/05/2022'),
 ('01/02/2022', '31/03/2022'),
 ('01/01/2022', '31/01/2022')]

In [7]:
cookie = 'Bifrost_SessionID=whmgz2fep3xyiidu0t0g3ief'

In [8]:

id_pattern = r"\b\d{8}\b"
ids = []
for start, end in dates_list:
    request_text = request_first_page(start, end)
    if request_text is not None:
        try:
            soup = BeautifulSoup(request_text, "html.parser")
            
            viewstate = soup.find("input", attrs={"type": "hidden", "name": "__VIEWSTATE", "id": "__VIEWSTATE"})["value"]
            eventvalidation = soup.find("input", attrs={"type": "hidden", "name": "__EVENTVALIDATION", "id": "__EVENTVALIDATION"})["value"]
            total_page = int(soup.find("span", attrs={"id": "ctl00_cplhContainer_lblTotalPage"}).string)
            
            if not viewstate or not eventvalidation or not total_page:
                raise ValueError(f"Missing data for request from {start} to {end}. Stopping.")
            
            # print(f"VIEWSTATE: {viewstate}, EVENTVALIDATION: {eventvalidation}, TOTAL_PAGE: {total_page}")
            else: 
                all_td = soup.find_all("td", style="display:none;")
                for td in all_td:
                    if re.fullmatch(id_pattern, td.text.strip()):
                        print(td.text.strip())
                        ids.append(td.text.strip())
                
                for page_index in range(2,total_page+1):
                    request_text_2 = request_more_page(eventvalidation,viewstate,page_index)
                    if request_text_2:                                  
                        try:
                            soup2 = BeautifulSoup(request_text_2,"html.parser")
                            all_td2 = soup2.find_all("td", style="display:none;")
                            for td2 in all_td2:
                                if re.fullmatch(id_pattern, td2.text.strip()):
                                    print(td2.text.strip())
                                    ids.append(td2.text.strip())
                        except Exception as e:
                            print(f"Error processing request from {start} to {end}, page{page_index}: {e}")
                            
        except Exception as e:
            print(f"Error processing request from {start} to {end}: {e}")

    else:
        print(f"Request failed for {start} to {end}.")


Request 01/10/2024,31/12/2024 thành công!
13544057
13543611
13543236
13542575
13534365
13534488
13533877
13525021
13536343
13525509
13525964
13525821
13525259
13524847
13524577
13523667
13517959
13517924
13283581
13283517
Request page 2 thành công!
13283386
13492295
13474024
13487887
13484064
13486536
13486240
13485627
13479733
13476682
13476461
13478771
13478664
13461491
13461070
13460870
13448515
13419668
13419621
13439710
Request page 3 thành công!
13439592
13432782
13432732
13424392
13424141
13423420
13422124
13426006
13422491
13422763
13399917
13399897
13399327
13388111
13387991
13381221
13385743
13380083
13379776
13348049
Request page 4 thành công!
13339307
13349309
13348800
13342302
13341746
13344525
13339330
13334809
13334723
13330768
13327376
13327344
13327312
13322080
13321118
13270324
13311702
13311585
13291802
13291774
Request page 5 thành công!
13292747
13292704
13292660
13254661
13254530
13257015
13233116
13234437
13232904
13219642
13224246
13198465
13205216
13205140
1318

KeyboardInterrupt: 

In [10]:
len(ids)

903

In [11]:
with open("output_co_ids.txt", "w", encoding="utf-8") as file:
    for item in ids:
        file.write(f"{item}\n")  # Mỗi ID trên một dòng